In [ ]:
pip install openai

In [1]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [4]:
import openai
from datetime import datetime, timedelta
import json

class DoctorAppointmentBot:
    def __init__(self, api_key):
        """Initialize the chatbot with OpenAI API key."""
        openai.api_key = api_key
        self.available_slots = self._generate_sample_slots()
        self.conversation_history = []

    def _generate_sample_slots(self):
        """Generate sample available appointment slots for next 7 days."""
        slots = {}
        start_date = datetime.now()
        for i in range(7):
            current_date = start_date + timedelta(days=i)
            date_str = current_date.strftime("%Y-%m-%d")
            slots[date_str] = [
                "09:00 AM", "10:00 AM", "11:00 AM",
                "02:00 PM", "03:00 PM", "04:00 PM"
            ]
        return slots

    def _get_chatgpt_response(self, user_input):
        """Get response from ChatGPT API."""
        self.conversation_history.append({"role": "user", "content": user_input})

        system_prompt = """
        You are a medical appointment scheduling assistant. Help patients schedule appointments
        while collecting necessary information like:
        - Patient name
        - Reason for visit
        - Preferred date and time
        - Contact information

        Be professional, courteous, and HIPAA-compliant. Don't share any patient information.
        Respond in a clear, structured way.

        Please consider the below doctor's name and their availability from Hyderabad .Also consider that all the below doctors are not available over the weekends.
        Doctor Name Hospital Name   Category    Availability
        Dr. Somnath Gupta   Yashodha Hospitals, Hitech City General Physician   9:00 am to 2:00 pm
        Dr. Tulika Tayal    Rainbow Children Hospital, Banjara Hills    Fetal Medicine Specialist   10:00 am to 1:00 pm
        Dr. Deepthi Kondagari   Endo Rheuma Care, Dilshukhnagar, Opposite RS Brothers   Endocrinologist 5:00 pm to 8:00 pm
        Dr. Sanjeev Kumar   Subodaya Luxury Appartments,YWCA Road, Abids    Cardiologist    8:30 am to 11:30 am
        Dr. Anudeep metropillar 1171, Blood Bank, 1st floor, IMS, opposite to Narayanaguda, near metro station, Narayanguda, Hyderabad, Telangana 500029    Homeopathy  10 am to 3:00 pm
        Dr Sushma Peruri    SUPRA GASTRO & SURGERY CLINIC, Malik Chambers, Old MLA Quarters Rd, opp. hotel Central Park, Avanti Nagar, Basheer Bagh, Hyderabad, Telangana 500029    Gastro, Piles, Fissure, Fistula and Breast lumps    24 hours
        Dr Pradeep Reddy M  Hyderabad MultiSpeciality Hospital, 16-2-674/11, Judges Colony, New Malakpet, Hyderabad, Telangana 500036   Orthopedic(Hip, Knee Joint Replacement) Surgeon 24 hours
        Dr Ravi Suman Reddy Dr Ravi Suman Reddy | Best NeuroSurgeon in Hyderabad | Best Spine Surgeon, Room 107 Yashoda Hospital, Raj Bhavan Rd, Matha Nagar, Somajiguda, Hyderabad, Telangana 500082   Neurologist 10:00 am to 4:00 pm
        Dr Chava Anjaneyulu Asian ENT Care Centre | Dr. Chava Anjaneyulu | ENT Doctor In Hyderabad | Best ENT Hospital In Hyderabad, 6-3-871 snehalatha, Greenlands Rd, near by pulla reddy sweets, beside Vivekananda hospital, Punjagutta Officers Colony, Begumpet, Hyderabad, Telangana 500016  ENT Specialist  9:00 am to 6:00 pm
        Dr Ravishankar Reddy    Ravi Speciality Clinic Flat no 202 Sri Krishna Sai LB towers Second floor Opposite SBI bank (yellareddyguda branch, Srinagar Colony, Hyderabad, Telangana 500073    Diabetologist   6:00 pm to 9:00 pm
        If the user has requested for an appointment then send the response in the below format:
        APPOINTMENT_REQUEST-- Doctor name, Hospital, Date and Time.
        As a AI assistant you need to always list all the Doctors quoted above.

        Please start an appointment scheduling dialogue using a structured JSON format. The first step should be a JSON prompt asking for essential appointment details: the name of the patient scheduling the appointment (Patient name), the name of the Doctor with whom the appointment needs to be fixed, the purpose or reason for visit of the appointment, preferred date and time and finally Contact information of the patient booking the appointment. The JSON structure should include a ‘complete’ status, which is initially false, a ‘prompt’ for the next piece of information only if required otherwise ‘prompt’ should be empty, and a ‘data’ section showing the current state of the collected information. Continue the conversation using JSON objects, updating and providing these JSON structures until all details are gathered. Conclude with a summary of the appointment details in the final JSON object, indicating ‘complete’: true. Do not provide any other response but the JSON formatted response in the format like the following. This example should be the first thing returned by you. Date should be in the format of mm-dd-yyyy. If given a date like “next Thursday” then confirm with some thing like “ Ok so that is January 14, 2025, correct? And what is… (the rest of the prompt)”. Do not accept any bookings in the past date. Do not ask for any information that would not fit in the provided json format. If they do not provide a complete or full patient name, then ask them to provide both first and last name. If no Contact information is given then prompt the patient to provide the email id or phone number. Without the Contact information filled, do not book any appointment.
        ‘’’{
          “complete”: false,
          “prompt”: “Please provide your name, the name of the Doctor you are meeting, the purpose of the appointment, its date and time, Contact Information(phone number or email).”,
          “data”: {
          “Name”: null,
          “purpose”: null,
          “date”: null,
          “time”: null
          "Contact details":null
          }
        }’’’
        """

        messages = [
            {"role": "system", "content": system_prompt},
            *self.conversation_history
        ]

        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=messages,
                max_tokens=500,
                temperature=0.7
            )
            assistant_response = response.choices[0].message.content
            self.conversation_history.append({"role": "assistant", "content": assistant_response})
            return assistant_response

        except Exception as e:
            return f"Error: {str(e)}"

    def check_availability(self, date_str):
        """Check available slots for a given date."""
        return self.available_slots.get(date_str, [])

    def book_appointment(self, date_str, time_str, patient_info):
        """Book an appointment for the given date and time."""
        if date_str in self.available_slots and time_str in self.available_slots[date_str]:
            self.available_slots[date_str].remove(time_str)
            return {
                "status": "success",
                "message": f"Appointment booked for {date_str} at {time_str}",
                "patient_info": patient_info
            }
        return {
            "status": "error",
            "message": "Selected slot is not available"
        }

    def process_user_input(self, user_input):
        """Process user input and return appropriate response."""
        # Get AI response
        ai_response = self._get_chatgpt_response(user_input)

        # If the AI response contains appointment request markers, process them
        #print(ai_response)
        if "APPOINTMENT_REQUEST" in ai_response:
            try:
                # Extract appointment details from AI response
                print("Extracting appointment details from AI response")
                #print("The Appointment request is "+ ai_response)
                request_details = json.loads(ai_response.split("APPOINTMENT_REQUEST")[1])


                date_str = request_details["date"]
                time_str = request_details["time"]
                patient_info = request_details["patient_info"]

                # Check availability and book appointment
                available_slots = self.check_availability(date_str)
                if time_str in available_slots:
                    booking_result = self.book_appointment(date_str, time_str, patient_info)
                    return booking_result["message"]
                else:
                    return f"Sorry, the requested slot is not available. Available slots for {date_str} are: {', '.join(available_slots)}"
            except json.JSONDecodeError:
                return ai_response

        return ai_response

def main():
    # Initialize the chatbot with your OpenAI API key
    api_key = ""
    bot = DoctorAppointmentBot(api_key)

    print("Welcome to the Doctor Appointment Chatbot!")
    print("Type 'quit' to exit")

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break

        response = bot.process_user_input(user_input)
        print(f"Bot: {response}")

if __name__ == "__main__":
    main()

Welcome to the Doctor Appointment Chatbot!
Type 'quit' to exit
You: list all the doctors at my place
Bot: ```json
{
  "complete": true,
  "prompt": "",
  "data": {
    "Doctors": [
      {
        "Name": "Dr. Somnath Gupta",
        "Hospital": "Yashodha Hospitals, Hitech City",
        "Category": "General Physician",
        "Availability": "9:00 am to 2:00 pm"
      },
      {
        "Name": "Dr. Tulika Tayal",
        "Hospital": "Rainbow Children Hospital, Banjara Hills",
        "Category": "Fetal Medicine Specialist",
        "Availability": "10:00 am to 1:00 pm"
      },
      {
        "Name": "Dr. Deepthi Kondagari",
        "Hospital": "Endo Rheuma Care, Dilshukhnagar, Opposite RS Brothers",
        "Category": "Endocrinologist",
        "Availability": "5:00 pm to 8:00 pm"
      },
      {
        "Name": "Dr. Sanjeev Kumar",
        "Hospital": "Subodaya Luxury Appartments,YWCA Road, Abids",
        "Category": "Cardiologist",
        "Availability": "8:30 am to 11:30 am